In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.3'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 128 kB in 2s (69.1 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep = ",", header = True, inferSchema = True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# add year column before creating temp view
from pyspark.sql.functions import year
df = df.withColumn("year", year(df['date']))
df.show(1)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
only showing top 1 row



In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('hs')

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
select round(avg(price),2) as Avg4BedroomPrice
from hs
where bedrooms like 4
"""
spark.sql(query).show()

+----------------+
|Avg4BedroomPrice|
+----------------+
|       299661.01|
+----------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
select round(avg(price),2) as AvgPrice, year
from hs
where bedrooms like 3 and bathrooms like 3
group by 2
order by 2
"""
spark.sql(query).show()

+---------+----+
| AvgPrice|year|
+---------+----+
|287287.82|2019|
|294204.16|2020|
|294211.46|2021|
|292725.69|2022|
+---------+----+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
select round(avg(price),2) as AvgPrice, year
from hs
where bedrooms like 3
and bathrooms like 3
and sqft_living >= 2000
group by 2
order by 2
"""
spark.sql(query).show()

+---------+----+
| AvgPrice|year|
+---------+----+
|294372.02|2019|
| 296314.6|2020|
|305118.97|2021|
|302871.28|2022|
+---------+----+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()

query = """
select round(avg(price),2) as AvgPrice, view
from hs
where price >= 350000
group by 2
order by 2 desc
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))
time_last = time.time() - start_time

+----------+----+
|  AvgPrice|view|
+----------+----+
| 1026669.5| 100|
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
+----------+----+
only showing top 20 rows

--- 0.9231984615325928 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table hs")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('hs')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
start_time = time.time()

query = """
select round(avg(price),2) as AvgPrice, view
from hs
where price >= 350000
group by 2
order by 2 desc
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))
print(f"That is {time_last - (time.time()-start_time)} seconds faster than last time!")
time_last = time.time() - start_time


+----------+----+
|  AvgPrice|view|
+----------+----+
| 1026669.5| 100|
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
+----------+----+
only showing top 20 rows

--- 0.9226715564727783 seconds ---
That is 0.0787355899810791 seconds faster than last time!


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("hs_partitioned")

In [ ]:
# 11. Read the parquet formatted data.
df_p = spark.read.parquet('hs_partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.
df_p.createOrReplaceTempView('hs_p')

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
start_time = time.time()

query = """
select round(avg(price),2) as AvgPrice, view
from hs_p
where price >= 350000
group by 2
order by 2 desc
"""
spark.sql(query).show()
if (time_last - (time.time() - start_time)) > 0:
  print(f"That is {time_last - (time.time()-start_time)} seconds faster than last time!")
else:
  print(f"That is {time_last - (time.time()-start_time)} seconds slower than last time.")
print("--- %s seconds ---" % (time.time() - start_time))

time_last = time.time() - start_time

+----------+----+
|  AvgPrice|view|
+----------+----+
| 1026669.5| 100|
|1061201.42|  99|
|1053739.33|  98|
|1129040.15|  97|
|1017815.92|  96|
| 1054325.6|  95|
| 1033536.2|  94|
|1026006.06|  93|
| 970402.55|  92|
|1137372.73|  91|
|1062654.16|  90|
|1107839.15|  89|
|1031719.35|  88|
| 1072285.2|  87|
|1070444.25|  86|
|1056336.74|  85|
|1117233.13|  84|
|1033965.93|  83|
| 1063498.0|  82|
|1053472.79|  81|
+----------+----+
only showing top 20 rows

That is -0.28012776374816895 seconds slower than last time.
--- 0.8756792545318604 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table hs")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('hs')

False

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
%cd /content/hs_partitioned

/content/hs_partitioned


In [33]:
!zip -r hs_partitioned.zip . hs_partitioned

	zip warning: name not matched: hs_partitioned
  adding: date_built=2016/ (stored 0%)
  adding: date_built=2016/part-00000-ac5493ff-8e10-4a00-bcd3-9232e3f192bb.c000.snappy.parquet (deflated 24%)
  adding: date_built=2016/.part-00000-ac5493ff-8e10-4a00-bcd3-9232e3f192bb.c000.snappy.parquet.crc (stored 0%)
  adding: date_built=2017/ (stored 0%)
  adding: date_built=2017/part-00000-ac5493ff-8e10-4a00-bcd3-9232e3f192bb.c000.snappy.parquet (deflated 24%)
  adding: date_built=2017/.part-00000-ac5493ff-8e10-4a00-bcd3-9232e3f192bb.c000.snappy.parquet.crc (stored 0%)
  adding: date_built=2014/ (stored 0%)
  adding: date_built=2014/part-00000-ac5493ff-8e10-4a00-bcd3-9232e3f192bb.c000.snappy.parquet (deflated 24%)
  adding: date_built=2014/.part-00000-ac5493ff-8e10-4a00-bcd3-9232e3f192bb.c000.snappy.parquet.crc (stored 0%)
  adding: date_built=2011/ (stored 0%)
  adding: date_built=2011/part-00000-ac5493ff-8e10-4a00-bcd3-9232e3f192bb.c000.snappy.parquet (deflated 24%)
  adding: date_built=2011/.p

In [34]:
from google.colab import files
files.download('hs_partitioned.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>